# Connect to DB

In [2]:
import pyodbc
import pandas as pd
import datetime

server = 'rpenterprise.database.windows.net'
database = 'PMAnalytics'
username = 'stephanie_lin'
password = 'GKfb5M0PDoJ0M7tVxa6Y'
driver= '{ODBC Driver 17 for SQL Server}'

# Construct connection string
conn_str = f'DRIVER={driver};SERVER=tcp:{server};PORT=1433;DATABASE={database};UID={username};PWD={password}'

# Connect to database
conn = pyodbc.connect(conn_str)

ProgrammingError: ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot open server 'rpenterprise' requested by the login. Client with IP address '174.64.24.197' is not allowed to access the server.  To enable access, use the Azure Management Portal or run sp_set_firewall_rule on the master database to create a firewall rule for this IP address or address range.  It may take up to five minutes for this change to take effect. (40615) (SQLDriverConnect); [42000] [Microsoft][ODBC Driver 17 for SQL Server]Invalid connection string attribute (0); [42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot open server 'rpenterprise' requested by the login. Client with IP address '174.64.24.197' is not allowed to access the server.  To enable access, use the Azure Management Portal or run sp_set_firewall_rule on the master database to create a firewall rule for this IP address or address range.  It may take up to five minutes for this change to take effect. 

In [3]:
# Execute the queries and load into dfs
sql_query1 = 'select * from work_effort'
work_effort = pd.read_sql(sql_query1, conn)

sql_query3 = 'select work_effort_id, EMPID, direct_reports, team_size, hire_date, grade from resource_view'
resource_view = pd.read_sql(sql_query3, conn)

sql_query2 = 'select * from work_effort_resource'
work_effort_resource = pd.read_sql(sql_query2, conn)

C:\Users\StephanieLin\AppData\Local\Temp\ipykernel_13632\1055891660.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  work_effort = pd.read_sql(sql_query1, conn)
C:\Users\StephanieLin\AppData\Local\Temp\ipykernel_13632\1055891660.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  resource_view = pd.read_sql(sql_query3, conn)
C:\Users\StephanieLin\AppData\Local\Temp\ipykernel_13632\1055891660.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  work_effort_resource = pd.read_sql(sql_query2, conn)


# Filter based on user input ; Merge into Master table

In [5]:
work_effort_roles = ['analyst', 'sponsor', 'manager', 'scrum_master', 'business_analyst', 'director']

# Display roles to the user
print("Please select from the following work effort roles (you can choose multiple by separating them with a comma):")
for i, role in enumerate(work_effort_roles, start=1):
    print(f"{i}. {role}")

# Get user input
user_input = input("Enter your choices (e.g., 1, 4): ")

selected_indices = [int(x.strip()) - 1 for x in user_input.split(",") if x.strip().isdigit()]
selected_roles = [work_effort_roles[i] for i in selected_indices if 0 <= i < len(work_effort_roles)]
        
if selected_roles:
    print("You selected:", ", ".join(selected_roles))
else:
    print("Not a valid entry; please try again!")


Please select from the following work effort roles (you can choose multiple by separating them with a comma):
1. analyst
2. sponsor
3. manager
4. scrum_master
5. business_analyst
6. director
Enter your choices (e.g., 1, 4): 1, 4
You selected: analyst, scrum_master


In [6]:
work_effort_types = ['Project', 'Program', 'Initiative']

print("Please select from the following work effort types (you can choose multiple by separating them with a comma):")
for i, role in enumerate(work_effort_types, start=1):
    print(f"{i}. {role}")

# Get user input
user_input = input("Enter your choices (e.g., 1, 2): ")

selected_indices = [int(x.strip()) - 1 for x in user_input.split(",") if x.strip().isdigit()]
selected_types = [work_effort_types[i] for i in selected_indices if 0 <= i < len(work_effort_types)]
        
if selected_types:
    print("You selected:", ", ".join(selected_types))
else:
    print("Not a valid entry; please try again!")


Please select from the following work effort types (you can choose multiple by separating them with a comma):
1. Project
2. Program
3. Initiative
Enter your choices (e.g., 1, 2): 2
You selected: Program


In [7]:
#CHANGE TO INPUT, maybe not needed
#cutoff_year = 2022
#cutoff_month = 12
#cutoff_day = 31
# = datetime.date(cutoff_year, cutoff_month, cutoff_day)

#date range filter
month_input = int(input("Enter the month to filter by (1-12): "))
work_effort['extract_date'] = pd.to_datetime(work_effort['extract_date'])

#assuming we are always just trying to ID employees
group_by = 'EMPID'

#filter work effort resource table by user selected role
filtered_wer = work_effort_resource[work_effort_resource['role'].isin(selected_roles)]

# Strip leading/trailing spaces
work_effort['rag_status'] = work_effort['rag_status'].str.strip()
work_effort['state'] = work_effort['state'].str.strip()
work_effort['work_effort_type'] = work_effort['work_effort_type'].str.strip()

#filter work effort table by other user selections
filtered_we = work_effort[
    (work_effort['state'].isin(['Committed', 'Approved'])) &
    (work_effort['rag_status'].isin(['Red', 'Amber', 'Green'])) &
#   (work_effort['most_current_end_date'] > cutoff_date) & 
    (work_effort['work_effort_type'].isin(selected_types)) &
    (work_effort['extract_date'].dt.month == month_input)
]

#work_effort.work_effort_type.unique()
#filtered_work_effort.head()
#filtered_we
#filtered_wer.role.unique()

#filtered_we.work_effort_type.unique()
#work_effort.work_effort_type.unique()


Enter the month to filter by (1-12): 4


In [8]:
#adding tenure columns. days will be default

# Assuming 'resource_view' DataFrame already exists and 'hire_date' is in datetime format
# If 'hire_date' is not in datetime format, you can convert it using:
resource_view['hire_date'] = pd.to_datetime(resource_view['hire_date'], errors='coerce')

# Calculate tenure in days and add it as a new column
today = pd.Timestamp.today()
resource_view['tenure'] = (today - resource_view['hire_date']).dt.days

# If you want tenure in years, you can divide by 365.25 (taking leap years into account)
#resource_view['tenure_years'] = resource_view['tenure'] / 365.25

#resource_view

In [9]:
#merge all 3 tables into df called 'master'
merged = pd.merge(filtered_we, filtered_wer, on='work_effort_id', how='left')
master = pd.merge(merged, resource_view, on='work_effort_id', how='left')
#pd.set_option('display.max_columns', None)
#master.head()

# Choose grouping calculation and column to group

In [26]:
# Assuming operation is a variable that contains the operation type ('count', 'average', or 'sum')
operation = input("Enter the operation (count, average, sum): ")

if operation.lower() == 'count':
    count_col = 'work_effort_id'  
    print(f"Counting based on: {count_col}")

elif operation.lower() == 'average':
    print("Choose a column for averaging:")
    print("1. Tenure")
    print("2. Grade")
    choice = input("Enter your choice (1 for Tenure, 2 for Grade): ")
    if choice == '1':
        avg_col = 'tenure'
    elif choice == '2':
        avg_col = 'grade'
    else:
        print("Invalid choice. Defaulting to 'Tenure'.")
        avg_col = 'tenure'
    print(f"Averaging based on: {avg_col}")

elif operation.lower() == 'sum':
    print("Choose a column for summing:")
    print("1. Direct_reports")
    print("2. Team_size")
    choice = input("Enter your choice (1 for Direct_reports, 2 for Team_size): ")
    if choice == '1':
        sum_col = 'direct_reports'
    elif choice == '2':
        sum_col = 'team_size'
    else:
        print("Invalid choice. Defaulting to 'Direct_reports'.")
        sum_col = 'direct_reports'
    print(f"Summing based on: {sum_col}")

else:
    print("Invalid operation specified. Please enter 'count', 'average', or 'sum'.")


Enter the operation (count, average, sum): sum
Choose a column for summing:
1. Direct_reports
2. Team_size
Enter your choice (1 for Direct_reports, 2 for Team_size): 2
Summing based on: team_size


In [11]:
def count_grouped_items(df, group_cols, count_col):
    if isinstance(group_cols, str):
        group_cols = [group_cols]
    grouped = df.groupby(group_cols)[count_col].count().reset_index(name=f'count_{count_col}')
    return grouped

def sum_grouped_items(df, group_cols, sumcol):
    if isinstance(group_cols, str):
        group_cols = [group_cols]
    grouped = df.groupby(group_cols)[sum_col].sum().reset_index(name=f'sum_{sum_col}')
    return grouped

def avg_grouped_items(df, group_cols, avgcol):
    if isinstance(group_cols, str):
        group_cols = [group_cols]
    grouped = df.groupby(group_cols)[avg_col].mean().reset_index(name=f'avg_{avg_col}')
    return grouped


In [12]:
if operation.lower() == 'count':
    df = count_grouped_items(master, group_by, count_col)

elif operation.lower() == 'average':
    master[avg_col] = pd.to_numeric(master[avg_col], errors='coerce')
    df = avg_grouped_items(master, group_by, avg_col)

elif operation.lower() == 'sum':
    master[sum_col] = pd.to_numeric(master[sum_col], errors='coerce')
    df = sum_grouped_items(master, group_by, sum_col)
else:
    print("Invalid operation specified. Please enter 'count', 'average', or 'sum'.")
    
df

,EMPID,avg_grade


# Calculate std dev and find EMPIDs to flag

In [13]:
def classify_deviation(df, metric_column, metric_prefix):
    avg_val = df[metric_column].mean()
    stddev_val = df[metric_column].std()
    df[f'{metric_prefix}_overall_stddev'] = stddev_val
    df[f'{metric_prefix}_overall_avg'] = avg_val

    df[f'{metric_prefix}_upper_threshold'] = avg_val + (3 * stddev_val)
    df[f'{metric_prefix}_lower_threshold'] = avg_val - (3 * stddev_val)

    def inner_classify_deviation(row):
        if row[metric_column] > row[f'{metric_prefix}_upper_threshold']:
            return f'Above +3 STD'
        elif row[metric_column] < row[f'{metric_prefix}_lower_threshold']:
            return f'Below -3 STD'
        else:
            return 'Within Range'

    df[f'{metric_prefix}_deviation_flag'] = df.apply(inner_classify_deviation, axis=1)
    return df


metric_column = df.columns[1]
metric_prefix = operation
deviation_counts = classify_deviation(df, metric_column, metric_prefix)
deviation_counts


,EMPID,avg_grade,average_overall_stddev,average_overall_avg,average_upper_threshold,average_lower_threshold,average_deviation_flag


In [14]:
#filter out not within range
result = deviation_counts[deviation_counts[operation+'_deviation_flag'] != 'Within Range']
#result

#get EMPIDs that are not within range
filtered_result = master[master['EMPID'].isin(result['EMPID'])]
#filtered_result.EMPID.unique()

In [15]:
list_to_flag = filtered_result.EMPID.unique()

list_to_flag = [x.strip() for x in list_to_flag]
#list_to_flag[0:10]

In [27]:
#if empid is in the above list for this work effort, add a column flagging that this is an issue on the original tables

# Add a new column 'Deviation_flag' with default value 'N'
resource_view['Deviation_flag'] = 'N'

# Flag rows where the EMPID is in list_to_flag
resource_view.loc[resource_view['EMPID'].isin(list_to_flag), 'Deviation_flag'] = 'Y'

resource_view
#LIST WHAT SPECIFIC KRI (grouping type and what the grouping is by) THIS IS A DEVIATION FLAG FOR
if operation.lower() == 'count':
    print(f"We have identified employees that have a {count_col} count of above 3 standard deviations.")

elif operation.lower() == 'average':
    print(f"We have identified employees that have an average {avg_col} of above 3 standard deviations.")

elif operation.lower() == 'sum':
    print(f"We have identified employees that have a total {sum_col} of above 3 standard deviations.")


We have identified employees that have a total team_size of above 3 standard deviations.
